In [155]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

In [156]:
model_type="knn"

In [157]:
def load_data(filename):
    data = np.load(filename)
    X=data[:,0:-1]
    y=data[:,-1]
    return X,y

In [158]:
#load train data
X_train, y_train = load_data("fashion_train.npy")

In [159]:
#scale data
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)

# perform PCA
pca = PCA(n_components=100)
pca.fit(X_train_scaled)
X_train_pca=pca.transform(X_train_scaled)

In [160]:
if model_type=="knn":
    #create KNN
    model=KNeighborsClassifier(n_neighbors=30)
elif model_type=="mlp":
    #create MLP
    model = MLPClassifier(solver='lbfgs', alpha=0.05,hidden_layer_sizes=(25,5, 2), random_state=1,max_iter=10_000)

model.fit(X_train_pca,y_train)

KNeighborsClassifier(n_neighbors=30)

In [161]:
#load test data
X_test, y_test = load_data("fashion_test.npy")

In [162]:
#scale test data
X_test_scaled=scaler.fit_transform(X_test)

#reduce dimensions of test data
X_test_pca=pca.transform(X_test_scaled)

In [163]:
#create predictions with model
preds = model.predict(X_test_pca)
acc = metrics.accuracy_score(y_test,preds)
cm = metrics.confusion_matrix(y_test,preds)
prec=metrics.precision_score(y_test,preds,average=None)
recall=metrics.recall_score(y_test,preds,average=None)
print(cm)
print(f"Accuracy: {acc}")
print(f"Precisions: {prec}")
print(f"Recalls: {recall}")

[[838   2   7  44 109]
 [  7 945  11  30   7]
 [ 16   0 833  19 132]
 [ 38   4  11 907  40]
 [204   1 158  32 605]]
Accuracy: 0.8256
Precisions: [0.75974615 0.99264706 0.81666667 0.87887597 0.6774916 ]
Recalls: [0.838 0.945 0.833 0.907 0.605]
